In [12]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import csv
import datetime

In [ ]:
# Open Street Map file
osm_file = open("melbourne_australia.osm", "r")

# Regular expression to find street type
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d" % (k, v)) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    # Parsing as XML
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

audit()

In [14]:
INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'


with open(INPUT_FILE, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames

        for row in reader:
            production_start_year = row['prductionStartYear']
            if type(row['productionStartYear']) != datetime.datetime:
                print(row)
            print(row['productionStartYear'])
def process_file(input_file, output_good, output_bad):

    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames

        for row in reader:
            if row['URI'][0:19] != 'http://dbpedia.org/':
                pass
            print(row['productionStartYear'])


    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in YOURDATA:
            writer.writerow(row)


<class 'str'>
OrderedDict([('URI', 'URI'), ('rdf-schema#label', 'http://www.w3.org/2000/01/rdf-schema#label'), ('rdf-schema#comment', 'http://www.w3.org/2000/01/rdf-schema#comment'), ('assembly_label', 'http://dbpedia.org/ontology/assembly'), ('assembly', 'http://dbpedia.org/ontology/assembly'), ('automobilePlatform_label', 'http://dbpedia.org/ontology/automobilePlatform'), ('automobilePlatform', 'http://dbpedia.org/ontology/automobilePlatform'), ('bodyStyle_label', 'http://dbpedia.org/ontology/bodyStyle'), ('bodyStyle', 'http://dbpedia.org/ontology/bodyStyle'), ('class_label', 'http://dbpedia.org/ontology/class'), ('class', 'http://dbpedia.org/ontology/class'), ('designCompany_label', 'http://dbpedia.org/ontology/designCompany'), ('designCompany', 'http://dbpedia.org/ontology/designCompany'), ('designer_label', 'http://dbpedia.org/ontology/designer'), ('designer', 'http://dbpedia.org/ontology/designer'), ('engine_label', 'http://dbpedia.org/ontology/engine'), ('engine', 'http://dbpedi